<a href="https://colab.research.google.com/github/choeuneheol/python-practice/blob/master/0907Word_Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP_Tutorial_03_Word_Embedding_2

In [ ]:
## Package 설치

 ## datasets(huggingface 자연어처리 dataset)
 ## konlpy (한글 자연어처리 Tool)
 ## tqdm (반복문의 Progress Bar)
 ## glove_python_binary (Glove 임베딩)

In [ ]:
!pip install datasets
!pip install konlpy
!pip install tqdm
!pip install glove_python_binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 3-0. 데이터셋 준비
 Naver sentiment Movie corpus(nsmc)
 
 영화 리뷰를 별점으로 기준으로 긍정(1), 부정(0)으로 구분한 데이터셋

In [ ]:
from datasets import load_dataset

#nsmc 데이터셋 다운로드
dataset = load_dataset("nsmc")

train_sents = [dataset["train"][idx]["document"] for idx in range(len(dataset["train"]))]
train_labels = [dataset["train"][idx]["label"] for idx in range(len(dataset["train"]))]
test_sents = [dataset["test"][idx]["document"] for idx in range(len(dataset["test"]))]
test_labels = [dataset["test"][idx]["label"] for idx in range(len(dataset["test"]))]

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from konlpy.tag import Okt
from tqdm import tqdm

# stopwords 정의
stopwords = ["하다","한","에","와","자","과","걍","잘","좀","는","의","가","이","은","들"]

okt = Okt()

train_nsmc_corpus = []
for sent_i in tqdm(train_sents):
  new_sent_i = sent_i.replace("[^가-힣ㄱ-하-ㅣ ]","")
  tokens = okt.morphs(sent_i, stem=True) #어간 추출
  tokens = [word_i for word_i in tokens if word_i not in stopwords]

  train_nsmc_corpus.append(tokens)

  test_nsmc_corpus = []
  for sent_i in tqdm(test_sents):
    new_sent_i = sent_i.replace("[^가-힣ㄱ-하-ㅣ ]","")
    tokens = okt.morphs(sent_i, stem=True)
    tokens = [word_i for word_i in tokens if word_i not in stopwords]

    test_nsmc_corpus.append(tokens)
# 자꾸 메모리가 부족하여 불러오질 못하고 있다...

# 3-1.Word2Vec (cBow)

word2Vec은 이웃 단어 예측을 통해 단어의 임베딩을 학습하는 기법


CBow 기법 / Skip-Gram  기법



CBow는 주위(Window) 단어를 보고 중심 단어를 예측

In [ ]:
from gensim.models import Word2Vec

#나는 학교에 지하철을 타고 간다
#나는 학교에 [      ] 타고 간다
#차, 버스, 지하철, 자전거
CBOW_model = Word2Vec(sentences=train_nsmc_corpus , size = 100 , window = 5 , min_count=5 , workers = 4 , sg =0)
# sg = 0 (CBOW) / sg = 1 (skip gram)

In [ ]:
CBOW_model.wv.most_similar("송강호", topn=5)

In [ ]:
# 사전에 없는 단어
CBOW_model.wv.most_similar("자연어처리", topn=5)

In [ ]:
CBOW_model.wv["송강호"]

In [ ]:
import numpy as np

def sentence_embedding_word2vec(model, sentence, dim=100):
  sent_emb = np.zeros(100)
  n=0

  for word_i in sentence:
    #Vocab에 존재하지 않는 단어의 경우 예외처리
    try:
      word_emb = model.wv[word_i]
      sent_emb += word_emb
      n += 1
    except:
      pass

    # 문장 임베딩 = 단어 임베딩의 평균
    if n ! = 0:
      sent_emb = sent_emb / n

    return sent_emb
  
train_x_cbow = list()
test_x_cbow = list()

for train_sent in tqdm(train_nsmc_corpus):
  train_x_cbow.append(sentence_embedding_word2vec(CBOW_model, train_sent))

for test_sent in tqdm(test_nsmc_corpus):
  test_x_cbow.append(sentence_embedding_word2vec(CBOW_model, test_sent))

In [ ]:
print(train_x_cbow[0])

# LogisticRegression

Linear Regression : 실수 범위의 값으로 예측


Logistic Regression : 0과 1사이의 값으로 예측

In [ ]:
from sklearn.linear_model import LogisticRegression
linear_model_cbow = LogisticRegression()
linear_model_cbow.fit(train_x_cbow, train_labels)

In [ ]:
train_pred = linear_model_cbow.predict(train_x_cbow)
print("train acc: {0}".format(np.mean(train_pred == train_labels)))
test_pred = linear_model_cbow.predict(test_x_cbow)
print("test acc: {0}".format(np.mean(test_pred == test_labels)))

# 3-2. Word2Vec (Skip-Gram)
Skip-Gram은 중심 단어를 보고 주위(Window) 단어들을 예측